In [7]:
# This notebook needs to be run in the root folder.
import pandas as pd
from spade.candidate_gen import generate_candidate_assertions
from spade.execute_assertions import execute_candidate_assertions
from spade.label_results import label_responses, prepare_for_optimizer
from spade.check_subsumes import evaluate_all_subsumes, collate_subsumption_results
from spade.optimizer import select_functions
# from rich import print

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
dataset_name = "negotiation"
from paper_experiments.negotiation.prompt_templates import TEMPLATES

prompt_template_strings = TEMPLATES
from paper_experiments.negotiation.examples import EXAMPLES

EXAMPLE = EXAMPLES[0]

print(f"There are {len(prompt_template_strings)} templates and {len(EXAMPLES)} examples.")

There are 8 templates and 23 examples.


In [ ]:
assertions = await generate_candidate_assertions(prompt_template_strings, EXAMPLE)

In [ ]:
# Print last prompt template
print(prompt_template_strings[-1])

In [ ]:
from paper_experiments.negotiation.candidate_assertions import ALL_FUNCTIONS
from paper_experiments.negotiation.examples import EXAMPLES


res = await execute_candidate_assertions(dataset_name, prompt_template_strings[-1], EXAMPLES, ALL_FUNCTIONS)


In [ ]:
prefix = f"/Users/shreyashankar/Documents/projects/promptdelta/paper_experiments/{dataset_name}"

In [ ]:
label_df = pd.read_csv(f"{prefix}/labeled_responses.csv")
print(label_df["label"].mean())

# Print stats
print(f"Num candidate functions: {len(ALL_FUNCTIONS)}")
print(f"Num good examples: {len(label_df[label_df['label'] == 1])}")
print(f"Num bad examples: {len(label_df[label_df['label'] == 0])}")

optimizer_dict = prepare_for_optimizer(res, label_df)

In [ ]:
response = res["response"].values[0]

K = await evaluate_all_subsumes(optimizer_dict["M"], ALL_FUNCTIONS, optimizer_dict["func_order"], prompt_template, response)

In [ ]:
K.sum() / (K.shape[0] * K.shape[1])

In [ ]:
# Pickle all the results
import inspect

path_name = f"/Users/shreyashankar/Documents/projects/promptdelta/paper_experiments/{dataset_name}/optimizer_input.pkl"

optimizer_dict["K"] = K
optimizer_dict["spade_functions"] = {func.__name__: inspect.getsource(func) for func in ALL_FUNCTIONS}
import pickle

with open(path_name, "wb") as f:
    pickle.dump(optimizer_dict, f)

In [ ]:
subsumption_df = collate_subsumption_results(optimizer_dict["M"], ALL_FUNCTIONS, optimizer_dict["func_order"], optimizer_dict["K"])
subsumption_df.to_csv(f"/Users/shreyashankar/Documents/projects/promptdelta/paper_experiments/{dataset_name}/subsumption_results.csv", index=False)

In [ ]:
from spade.check_subsumes import sample_subsumption_prompts_and_results

subsumption_sample_llm_labeled = await sample_subsumption_prompts_and_results(optimizer_dict["M"], ALL_FUNCTIONS, optimizer_dict["func_order"], optimizer_dict["K"], prompt_template_strings[-1], response)
subsumption_sample_llm_labeled.to_csv(f"/Users/shreyashankar/Documents/projects/promptdelta/paper_experiments/subsumption/data/{dataset_name}.csv", index=False)

In [ ]:
# Solve the optimization problem

optimizer_res = select_functions(path_name, tau=0.25, alpha=0.6)

# Pickle the results
import pickle
with open(f"/Users/shreyashankar/Documents/projects/promptdelta/paper_experiments/{dataset_name}/optimizer_results.pkl", "wb") as f:
    pickle.dump(optimizer_res, f)


In [ ]:
# Visualize performance

# Turn into DF
import pandas as pd

optimizer_res_df = []
for method in ["spade_base", "spade_cov", "spade_sub"]:
    optimizer_res_df.append({"method": method, "ffr": optimizer_res[method]["ffr"], "example failure coverage": optimizer_res[method]["coverage"], "frac_functions_selected": optimizer_res[method]["frac_functions_selected"], "frac_non_subsumed_excluded_functions": optimizer_res[method]["frac_non_subsumed_excluded_functions"]})
    
optimizer_res_df = pd.DataFrame(optimizer_res_df)
print(optimizer_res_df)


## Evaluating subsumption with one prompt (optimizing for precision)

In [ ]:
from spade.check_subsumes import identify_subsumption_pairs
from paper_experiments.negotiation.candidate_assertions import ALL_FUNCTIONS
import pickle

dataset_name = "negotiation"
path_name = f"/Users/shreyashankar/Documents/projects/promptdelta/paper_experiments/{dataset_name}/optimizer_input.pkl"
with open(path_name, "rb") as f:
    optimizer_dict = pickle.load(f)
K_and_pretty_df = await identify_subsumption_pairs(optimizer_dict["M"], ALL_FUNCTIONS, optimizer_dict["func_order"])

In [ ]:
K_and_pretty_df["K"].sum() / (K_and_pretty_df["K"].shape[0] * K_and_pretty_df["K"].shape[1])

In [ ]:
# Pickle all the results
import inspect
from spade.optimizer import select_functions

path_name = f"/Users/shreyashankar/Documents/projects/promptdelta/paper_experiments/{dataset_name}/optimizer_input.pkl"

optimizer_dict["K"] = K_and_pretty_df["K"]
optimizer_dict["spade_functions"] = {func.__name__: inspect.getsource(func) for func in ALL_FUNCTIONS}
import pickle

with open(path_name, "wb") as f:
    pickle.dump(optimizer_dict, f)
    
# Dump subsumption df
K_and_pretty_df["human_readable_results"].to_csv(f"/Users/shreyashankar/Documents/projects/promptdelta/paper_experiments/{dataset_name}/subsumption_results.csv", index=False)

# Solve the optimization problem
optimizer_res = select_functions(path_name, tau=0.25, alpha=0.6)

# Turn into DF
import pandas as pd

optimizer_res_df = []
for method in ["spade_base", "spade_cov", "spade_sub"]:
    optimizer_res_df.append({"method": method, "ffr": optimizer_res[method]["ffr"], "example failure coverage": optimizer_res[method]["coverage"], "frac_functions_selected": optimizer_res[method]["frac_functions_selected"], "frac_non_subsumed_excluded_functions": optimizer_res[method]["frac_non_subsumed_excluded_functions"]})
    
optimizer_res_df = pd.DataFrame(optimizer_res_df)
print(optimizer_res_df)

# Pickle the results
# import pickle
# with open(f"/Users/shreyashankar/Documents/projects/promptdelta/paper_experiments/{dataset_name}/optimizer_results.pkl", "wb") as f:
#     pickle.dump(optimizer_res, f)